In [1]:
import os
import sys
import random
import json
import importlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime

tf.config.run_functions_eagerly(True)

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config 
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
import mrcnn_code.egg as egg

from mrcnn_code.egg import EggDataset

# importlib.reload(modellib)

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "models")
MODEL_PATH = os.path.join(MODEL_DIR, ".weights.h5")

2025-06-18 13:34:42.772445: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-18 13:34:42.787527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750246482.804429    2944 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750246482.808542    2944 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750246482.822830    2944 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [1]:
import tensorflow as tf

tf.config.run_functions_eagerly(True)
print("Eager mode aktiv:", tf.executing_eagerly())

# Test: funktioniert das hier?
ds = tf.data.Dataset.from_tensor_slices([1, 2, 3])
for x in ds:
    print("Ein Element:", x)

2025-06-18 13:30:42.561428: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-18 13:30:42.574055: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750246242.593202    2789 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750246242.598446    2789 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750246242.609066    2789 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Eager mode aktiv: True
Ein Element: tf.Tensor(1, shape=(), dtype=int32)
Ein Element: tf.Tensor(2, shape=(), dtype=int32)
Ein Element: tf.Tensor(3, shape=(), dtype=int32)


2025-06-18 13:30:45.337517: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-06-18 13:30:45.407235: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [2]:
EGG_DIR = os.path.join(ROOT_DIR, "Data/egg_annot")
config = egg.EggConfig()
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)
model.load_weights(MODEL_PATH, by_name=True)
dataset_train = EggDataset()
dataset_train.load_egg(EGG_DIR, "train")
dataset_train.prepare()

dataset_val = EggDataset()
dataset_val.load_egg(EGG_DIR, "val")
dataset_val.prepare()

In [3]:
model.train(dataset_train,
            dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=20,
            layers='heads')

/home/tibor/Documents/python/mosquito-egg-identification/venv/lib/python3.12/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


False
>>> Typ train_dataset: <class 'tensorflow.python.data.ops.flat_map_op._FlatMapDataset'>
>>> Typ val_dataset: <class 'tensorflow.python.data.ops.flat_map_op._FlatMapDataset'>
>>> steps_per_epoch: 100 <class 'int'>
>>> validation_steps: 50 <class 'int'>


RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.

In [5]:
inputs, outputs = next(iter(modellib.DataGenerator))
print(type(inputs))   # sollte <class 'list'>
print(type(outputs))  # sollte <class 'dict'>


TypeError: 'type' object is not iterable

In [11]:
# Zuerst im Inferenz-Modus erstellen
inference_config = egg.EggConfig()
inference_config.NAME = "egg_inference"
model = modellib.MaskRCNN(mode="inference", 
                         config=inference_config,
                         model_dir=MODEL_DIR)

# Dann auf Training umstellen
model.keras_model.trainable = True
model.set_trainable(layer_regex="all")

Selecting layers to train


In [12]:
import h5py
import numpy as np
from mrcnn.model import ProposalLayer, PyramidROIAlign  # Import der Custom Layers

COCO_MODEL_PATH = os.path.join(MODEL_DIR, "mask_rcnn_coco.h5")

def load_coco_weights_with_mismatch(model, coco_path):
    # Layer, die trotz Shape-Unterschied übertragen werden können
    TRANSFERABLE_LAYERS = [
        'mrcnn_class_conv1', 'mrcnn_class_bn1',
        'mrcnn_class_conv2', 'mrcnn_class_bn2',
        'mrcnn_mask_conv1', 'mrcnn_mask_bn1',
        'mrcnn_mask_conv2', 'mrcnn_mask_bn2',
        'mrcnn_mask_conv3', 'mrcnn_mask_bn3',
        'mrcnn_mask_conv4', 'mrcnn_mask_bn4'
    ]
    
    with h5py.File(coco_path, 'r') as f:
        layer_names = [n.decode('utf8') for n in f.attrs['layer_names']]
        
        for layer in model.keras_model.layers:
            if layer.name in layer_names:
                try:
                    g = f[layer.name]
                    weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
                    weights = [np.array(g[wn]) for wn in weight_names]
                    
                    # Spezialbehandlung für bestimmte Layer
                    if layer.name in TRANSFERABLE_LAYERS:
                        print(f"✓ {layer.name} (transferred despite shape difference)")
                        layer.set_weights(weights)
                    elif layer.name == 'mrcnn_class_logits':
                        # Nur die ersten N Gewichte übertragen
                        new_weights = [w[:model.config.NUM_CLASSES] if w.ndim > 0 else w for w in weights]
                        layer.set_weights(new_weights)
                        print(f"✓ {layer.name} (partial transfer)")
                    else:
                        if [w.shape for w in weights] == [w.shape for w in layer.get_weights()]:
                            layer.set_weights(weights)
                            print(f"✓ {layer.name}")
                        else:
                            print(f"✗ {layer.name} (shape mismatch)")
                except Exception as e:
                    print(f"⚠ {layer.name} error: {str(e)}")
            else:
                print(f"☐ {layer.name} (not in COCO)")

# Verwendung:
load_coco_weights_with_mismatch(model, COCO_MODEL_PATH)

✓ input_image
✓ zero_padding2d_1
✓ conv1
✓ bn_conv1
✓ activation_74
✓ max_pooling2d_1
✓ res2a_branch2a
✓ bn2a_branch2a
☐ activation_75 (not in COCO)
✓ res2a_branch2b
✓ bn2a_branch2b
☐ activation_76 (not in COCO)
✓ res2a_branch2c
✓ res2a_branch1
✓ bn2a_branch2c
✓ bn2a_branch1
✓ add_33
✓ res2a_out
✓ res2b_branch2a
✓ bn2b_branch2a
☐ activation_77 (not in COCO)
✓ res2b_branch2b
✓ bn2b_branch2b
☐ activation_78 (not in COCO)
✓ res2b_branch2c
✓ bn2b_branch2c
☐ add_34 (not in COCO)
✓ res2b_out
✓ res2c_branch2a
✓ bn2c_branch2a
☐ activation_79 (not in COCO)
✓ res2c_branch2b
✓ bn2c_branch2b
☐ activation_80 (not in COCO)
✓ res2c_branch2c
✓ bn2c_branch2c
☐ add_35 (not in COCO)
✓ res2c_out
✓ res3a_branch2a
✓ bn3a_branch2a
☐ activation_81 (not in COCO)
✓ res3a_branch2b
✓ bn3a_branch2b
☐ activation_82 (not in COCO)
✓ res3a_branch2c
✓ res3a_branch1
✓ bn3a_branch2c
✓ bn3a_branch1
☐ add_36 (not in COCO)
✓ res3a_out
✓ res3b_branch2a
✓ bn3b_branch2a
☐ activation_83 (not in COCO)
✓ res3b_branch2b
✓ bn3b_bra

In [23]:
model.keras_model.save('model.keras')

In [25]:
model.keras_model.save_weights('.weights.h5')

In [20]:
layers_to_train = [
    'mrcnn_class_logits',
    'mrcnn_bbox_fc',
    'mrcnn_mask',
    'rpn_class',
    'rpn_bbox'
]

# Für jeden Layer einzeln
for layer_pattern in layers_to_train:
    model.set_trainable(layer_pattern)

Selecting layers to train
mrcnn_class_logits     (TimeDistributed)
Selecting layers to train
mrcnn_bbox_fc          (TimeDistributed)
Selecting layers to train
mrcnn_mask             (TimeDistributed)
Selecting layers to train
Selecting layers to train
